[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/spatial.ipynb)


# Spatial

:::{note}

Please see: 

- {doc}`docs:vitessce`
- {doc}`docs:rxrx`

This page is for now just a stub.

:::

In [ ]:
# !pip install 'lamindb[jupyter,bionty]'
!lamin init --storage ./test-spatial --schema bionty

In [ ]:
import lamindb as ln
import bionty as bt
import matplotlib.pyplot as plt
import scanpy as sc

ln.context.uid = "daeFs3PkquDW0000"
ln.context.track()

## An example spatial dataset

Here, we have a spatial gene expression dataset measured using [Visium](https://www.10xgenomics.com/products/spatial-gene-expression) from [Suo22](https://developmental.cellatlas.io/fetal-immune).

This collection contains two parts:

1. a high-res image of a slice of fetal liver
2. a single cell expression dataset in .h5ad

In [ ]:
img_path = ln.core.datasets.file_tiff_suo22()
img = plt.imread(img_path)
plt.imshow(img)
plt.show()

In [ ]:
adata = ln.core.datasets.anndata_suo22_Visium10X()
# subset to the same image
adata = adata[adata.obs["img_id"] == "F121_LP1_4LIV"].copy()
adata

In [ ]:
# plot where CD45+ leukocytes are in the slice
sc.pl.scatter(adata, "array_row", "array_col", color="ENSG00000081237")

## Validate annotations

We'll register the single-cell data and the image as a `Collection`.

In [ ]:
curate = ln.Curator.from_anndata(
    adata,
    var_index=bt.Gene.ensembl_gene_id,
    categoricals={"sample": ln.ULabel.name},
    organism="human",
)

In [ ]:
curate.validate()

In [ ]:
curate.add_validated_from_var_index()

In [ ]:
curate.add_new_from("sample")

In [ ]:
curate.validate()

## Register curated artifact

In [ ]:
artifact_ad = curate.save_artifact(description="Suo22 Visium10X image F121_LP1_4LIV")

In [ ]:
artifact_ad.describe()

## Register a collection

In [ ]:
artifact_img = ln.Artifact(img_path, description="Suo22 image F121_LP1_4LIV")
artifact_img.save()

In [ ]:
collection = ln.Collection([artifact_ad, artifact_img], name="Suo22")
collection.save()

In [ ]:
# clean up test instance
!lamin delete --force test-spatial
!rm -r test-flow